In [1]:
#!pip install imdbpie

In [2]:
import os
import subprocess
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
import requests
from imdbpie import Imdb
import nltk
import matplotlib.pyplot as plt
import urllib
from bs4 import BeautifulSoup
import nltk
import collections
import re
import csv
import psycopg2

import scrapy
from scrapy.selector import Selector
from scrapy.http import HtmlResponse

%matplotlib inline

In [3]:
imdb = Imdb(anonymize=True)

In [4]:
top = imdb.top_250()
df = pd.DataFrame(top)

In [5]:
df.head()

,can_rate,image,num_votes,rating,tconst,title,type,year
0,True,{u'url': u'https://images-na.ssl-images-amazon...,1715518,9.3,tt0111161,The Shawshank Redemption,feature,1994
1,True,{u'url': u'https://images-na.ssl-images-amazon...,1172561,9.2,tt0068646,The Godfather,feature,1972
2,True,{u'url': u'https://images-na.ssl-images-amazon...,803421,9.0,tt0071562,The Godfather: Part II,feature,1974
3,True,{u'url': u'https://images-na.ssl-images-amazon...,1701921,9.0,tt0468569,The Dark Knight,feature,2008
4,True,{u'url': u'https://images-na.ssl-images-amazon...,456533,8.9,tt0050083,12 Angry Men,feature,1957


In [6]:
df['rating'].value_counts()

8.1    70
8.3    45
8.2    44
8.4    24
8.5    24
8.6    15
8.7     7
8.0     7
8.8     5
8.9     5
9.0     2
9.2     1
9.3     1
Name: rating, dtype: int64

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 8 columns):
can_rate     250 non-null bool
image        250 non-null object
num_votes    250 non-null int64
rating       250 non-null float64
tconst       250 non-null object
title        250 non-null object
type         250 non-null object
year         250 non-null object
dtypes: bool(1), float64(1), int64(1), object(5)
memory usage: 14.0+ KB


In [8]:
df['type'].value_counts()

feature    250
Name: type, dtype: int64

I got the movie id from here: http://www.imdb.com/title/tt0468569/

In [9]:
#FOUND SOURCE CODE FOR IMDBPIE https://github.com/richardasaurus/imdb-pie/blob/master/src/imdbpie/imdbpie.py
#FROM HERE I EMULATED THE TOP_250() FUNCTION TO SCRAPE THE BOTTOM 100 USING THE SITE http://www.imdb.com/chart/bottom 
#PROVIDED IN THE README FILE.  

#  def top_250(self):
#         url = self._build_url('/chart/top', {})
#         response = self._get(url)
#         return response['data']['list']['list']

In [10]:
class Imdb(Imdb):
    def bottom(self):
        url = self._build_url('/chart/bottom', {})
        response = self._get(url)
        return response['data']['list']['list']

In [11]:
imdb_bottom = Imdb(anonymize=True)

In [12]:
bottom = imdb_bottom.bottom()
df1 = pd.DataFrame(bottom)

In [13]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 8 columns):
can_rate     100 non-null bool
image        100 non-null object
num_votes    100 non-null int64
rating       100 non-null float64
tconst       100 non-null object
title        100 non-null object
type         100 non-null object
year         100 non-null object
dtypes: bool(1), float64(1), int64(1), object(5)
memory usage: 5.6+ KB


In [14]:
Movies = df.append(df1, ignore_index=True)

In [15]:
Movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 350 entries, 0 to 349
Data columns (total 8 columns):
can_rate     350 non-null bool
image        350 non-null object
num_votes    350 non-null int64
rating       350 non-null float64
tconst       350 non-null object
title        350 non-null object
type         350 non-null object
year         350 non-null object
dtypes: bool(1), float64(1), int64(1), object(5)
memory usage: 19.6+ KB


In [16]:
reviews=imdb.get_title_reviews("tt0468569", max_results=15)

In [17]:
for rev in reviews:
    print '-------------------------'
    print rev.text
    print '\nmy rating is', rev.rating
    print '--------------------------'

-------------------------
We've been subjected to enormous amounts of hype and marketing for the Dark Knight. We've seen Joker scavenger hunts and one of the largest viral campaigns in advertising history and it culminates with the actual release of the movie.

Everything that's been said is pretty much spot on. This is the first time I can remember where a summer blockbuster film far surpasses the hype.

For as much action as there is in this movie, it's the acting that makes it a great piece of work. Between all the punches, explosions and stunt-work is some great dialog work. All the actors have their moments.

Bale's Batman is the definitive Batman because we see everything in this character finally on film. Martial arts skills, cunning, great tactical thinking, forensic application, technological genius to advance or improve Luscious Fox's inventions/technological breakthroughs, intimidating personality, and even a little swashbuckling. 

As for Heath, yes he gets credit for his p

In [18]:
#CALLING IN NLTK STEMMER TO CREATE A FUNCTION WHICH COMBINES A STEMMER WITH A TOKENIZER IN ORDER TO USE WITH THE 
#TFIDFVECTORIZER 

In [19]:
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def stem_tokens(tokens, stemmer):
    stemmed = []
    for item in tokens:
        stemmed.append(stemmer.stem(item))
    return stemmed

def tokenize(text):
    tokens = nltk.word_tokenize(text)
    stems = stem_tokens(tokens, stemmer)
    return stems

In [ ]:
#BEGINNING TO USE IMDB RATINGS ALONG WITH REVIEWS TO CREATE NEW DATAFRAMES, WHERE 
#REVIEWS ARE STRINGS OF 15 REVIEWS.  

In [ ]:
#ADDED THE TRY AND EXCEPT STATEMENTS TO GATHER DATA FROM IMDB WEBSITE 

review_list = []
review_rating_as_int = []

#THE MOVIE_DICT IS GOING TO BE USED ON MY SIDE PROJECT.  PRINTING OUT THE MOST COMMON WORDS FOR EACH MOVIE TITLE
#movie_dict = {}

for i in range(len(Movies['tconst'])):
    for rev in reviews:
        try:
            reviews=imdb.get_title_reviews(i, max_results=15)   
        
            review_list.append(rev.text)  
            
            review_rating_as_int.append(rev.rating)
            #movie_dict[Movie['title'][i] = rev.text]
            print rev.rating
        except:
            pass


In [ ]:
#SIDE PROJECT. PRODUCING SIMPLE WORD COUNT FOR OUT MOVIES.  I WANT TO SEE THE MOST POPULAR 
#WORDS USED, PER TITLE, FROM THE REVIEWS. 

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cvec = CountVectorizer(strip_accents='ascii', stop_words='english')

In [ ]:
cvec.fit(review_list)

In [ ]:
common_words = []
for i in range(len(review_list)):
    X_temp = pd.DataFrame(cvec.transform(rev_list).todense(),
                          columns=cvec.get_feature_names())
    word_counts = X_temp.sum(axis=0)
    
    
    print Movies['title'][i], "most common words"
    cw = word_counts.sort_values(ascending= False).head(10)
    print cw
    common_words.extend(cw.index)

In [ ]:
#END OF WORD CLOUD SIDEPROJECT

In [ ]:
#CREATING A NEW DATAFRAME OF THE READ IN DATA.  REVIEW IS THE HEADER FOR MY COLUMNS WHICH
#CONTAINS 15 REVIEWS PER MOVIE 

#RATING IS THE COLUMN WHICH CONTAINS EACH RATING FOR EACH REVIEW FED IN 

In [ ]:
pie = pd.DataFrame({'review': review_list, 'rating': review_rating_as_int})

In [ ]:
#DROPPPING ALL NA'S AND NAN VALUES FROM OUR DATASET.  SOME REVIEWS SIMPLY DID NOT COME WITH
#A RATING.  BUT WE KNOW, FROM THE SECTION THEY WERE PULLED IF THEY WERE GOOD OR BAD 

In [ ]:
pie = pie.dropna()

In [ ]:
#USING A CUTOFF HERE TO CREATE A TARGET COLUMN.  ALL RATINGS 8 AND HIGHER ARE CONSIDERED 
#GOOD, WHILE THE THE REST I AM CONSIDERING BAD

In [ ]:
from sklearn import preprocessing 

y = pie['rating']
y = LabelEncoder.fit()

In [ ]:
X = pie['review']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
#DO I NEED TO MAKE A TODENSE() FUNCTION CALL HERE 

In [ ]:
tvec.fit(X_train)

In [ ]:
X_train_features = tvec.transform(X_train).todense()
X_test_features  = tvec.transform(X_test).todense()

In [ ]:
#CREATING A CROSS VALIDATION TO RUN WITH A DECISION TREE AND BAGGCLASSIFIER

In [ ]:
from sklearn.cross_validation import cross_val_score, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier

cv = StratifiedKFold(y_train, n_folds=5, shuffle=True, random_state=41)

In [ ]:
dt = DecisionTreeClassifier(class_weight='balanced')
s = cross_val_score(dt, X_train_features, y_train, cv=cv, n_jobs=-1)
print "{} Score:\t{:0.3} ± {:0.3}".format("Decision Tree", s.mean().round(3), s.std().round(3))

In [ ]:
from sklearn.ensemble import BaggingClassifier 
rf = RandomForestClassifier(class_weight='balanced', n_jobs=-1)

for n_est in [10,20,30,40,60,80]:
    bagg = BaggingClassifier( base_estimator=rf, n_estimators=n_est, bootstrap_features=True)
    
    print 'number of estimators: ',n_est, '5x CV score: ', cross_val_score(bagg,X_train_features,
                                                                           y_train, cv=cv).mean()

In [ ]:
#FROM HERE I WILL CHOOOSE THE BEST NUMBER OF ESTIMATORS TO RUN HERE.  MAYBE LOOK AT 
#GRIDSEARCHING 

In [ ]:
bagg = BaggingClassifier(base_estimator=rf, n_estimators=80, bootstrap_features=True)

In [ ]:
bagg.fit(X_train_features,y_train)

In [ ]:
Ypred = bagg.predict(X_test_features)

In [ ]:
Ypred

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [ ]:
print confusion_matrix(y_test, Ypred)

In [ ]:
print classification_report(y_test, Ypred)

In [ ]:
GOING TO USE GRIDSEARCH HERE.  IT SEEMS THE LOGISTIC REGRESSION HAS WORKED BEST SO GOING TO TUNE THE MODEL WITH A 
GRID SEARCH OVER SEVERAL LOGIST REGRESSION PARAMETERS. 

from sklearn.grid_search import GridSearchCV

logreg_parameters = {
    'penalty':['l1','l2'],
    'C':np.logspace(-5,1,50),
    'solver':['liblinear']
}

log_gs = GridSearchCV(LogisticRegression(), logreg_parameters, cv=cv, verbose=1)


print('GRID SEARCH BEST SCORE: %.4f' % logs_gs.best_score_)
print('GRID SEARCH BEST PARAMER FOR LOGRED: ')
print log_gs.best_params_

In [ ]:
log_gs.fit(X_train_features, y_train)
Y_pred_log = log_gs.predict(X_test_features)

In [ ]:
print confusion_matrix(y_test, Y_pred_log)

In [ ]:
print classification_report(y_test, Y_pred_log)

In [ ]:
# BEGINNING NEW SCRAPE OF IMDB WEBSITE IN ORDER TO GET MORE DATA TO TEST THE MODEL

#BRINGING IN DATA FOR MOVIES WITH GOOD RATINGS 
#http://www.imdb.com/search/title?year=2016,2016&title_type=feature&sort=user_rating,desc


#BRINGING IN DATA FOR MOVIES WITH BAD RATINGS 
#http://www.imdb.com/search/title?year=2016,2016&title_type=feature&sort=user_rating,asc


#IMPORTING BEAUTIFUL SOUP AND REQUESTS TO READ IN OUR DATA
from bs4 import BeautifulSoup
import requests


good_2016_movies_url = 'http://www.imdb.com/search/title?year=2016,2016&title_type=feature&sort=user_rating,desc'
good_2016_movies_html = requests.get(good_2016_movies_url)

# so now we have the html
# but we need a nice way to search the html

#good_2016_movies_html.text

good_2016_movies_soup = BeautifulSoup(good_2016_movies_html.text ,'html.parser')

good_2016_movies_div = good_2016_movies_soup.findAll("div", { "class" : "ribbonize" })

#EXPECTING 50 RESULTS HERE 
len(good_2016_movies_div)

good_2016_movies_div[0]

#CREATING DATA FRAME FOR TCONSTS FOR EACH MOVIE
type(good_2016_movies_div[0])

#TURNING OUR NEW DATA FROM BS4.ELEMENT.TAG OBJECT INTO SOMETHING I CAN USE.  

good_2016_movies_div[0]['data-tconst']

good_2016_movies_tconst = []
for div_elm in good_2016_movies_div:
    good_2016_movies_tconst.append(div_elm['data-tconst'])

#RUNNING THE SAME PROCESS FOR A LIST OF BAD MOVIES 
bad_movies_2016_url = 'http://www.imdb.com/search/title?year=2016,2016&title_type=feature&sort=user_rating,asc'
bad_movies_2016_html = requests.get(bad_movies_2016_url)

bad_movies_2016_soup = BeautifulSoup(bad_movies_2016_html.text, 'html.parser')

bad_movies_2016_div = bad_movies_2016_soup.findAll("div", {"class": "ribbonize"})

bad_2016_movies_tconst = []
for div_elm in bad_movies_2016_div:
    bad_2016_movies_tconst.append(div_elm['data-tconst'])

all_2016_tconst = good_2016_movies_tconst[:5] + bad_2016_movies_tconst

reviews_2016_as_text = []
review_2016_rating_as_int = []

for tconst in all_2016_tconst:
    try:
        reviews_for_movie_i = imdb.get_title_reviews(tconst, max_results=2)
        if reviews_for_movie_i is None:
            continue
        for movie in reviews_for_movie_i:
            reviews_2016_as_text.append(movie.text)
            review_2016_rating_as_int.append(movie.rating)
    except:
        print 'werent able to get movie', tconst

print len(reviews_2016_as_text)
print len(review_2016_rating_as_int)


df_2016_movies = pd.DataFrame({'reviews': reviews_2016_as_text , 'ratings': review_2016_rating_as_int})

df_2016_movies = df_2016_movies.dropna()

In [ ]:
#SO VECTORIZE FIT ETC. X1 AND SUBSTITUTE FOR BAGG.PREDICT(X_TEST)

In [ ]:
X1 = df_2016_movies['reviews']

In [ ]:
X1_features = tvec.transform(X1).todense()

In [ ]:
y1= df_2016_movies['reviews']

In [ ]:
y1 = LabelEncoder.fit()

In [ ]:
Y_pred_test = log_gs.predict(X1_features)

In [ ]:
#PRINTING MATRIX TO SEE HOW WELL MY MODEL HAS DONE.

In [ ]:
print confustion_matrix(y, Y_pred_test)

In [ ]:
print classification_report(y, Y_pred_test)